In [ ]:
import joblib
import pandas as pd
import numpy as np
from typing import Dict, List

# --- 1. 저장된 모델 및 임계값 로드 (경로를 'models/'로 가정) ---
# NOTE: 실제 파일 경로에 따라 'models/' 부분을 수정해야 합니다.
try:
    LOADED_MODEL = joblib.load('models/final_churn_model.pkl')
    LOADED_THRESHOLDS = joblib.load('models/risk_thresholds.pkl')
    # 학습된 모델의 피처 목록 (컬럼 이름과 순서)
    # XGBoost/GradientBoostingClassifier는 'feature_names_in_' 속성을 가집니다.
    MODEL_FEATURES: List[str] = list(LOADED_MODEL.feature_names_in_) 
    
except FileNotFoundError:
    print("오류: 모델 파일(final_churn_model.pkl 또는 risk_thresholds.pkl)을 'models/' 경로에서 찾을 수 없습니다.")
    print("파일 경로를 확인하거나, 'model_pipeline.py'를 먼저 실행하여 파일을 생성해야 합니다.")
    raise

# --- 2. 전처리 함수 수정 및 완성 ---

HIGH_IMPORTANCE_FEATURES = [
    'marketing_open_rate_6m', 'tenure_months', 'complaints_6m', 'age', 
    'spent_change_ratio',
    'login_m1', 'login_m2', 'login_m3',
    'spent_m1', 'spent_m2', 'spent_m3', 
    'txn_m1', 'txn_m2', 'txn_m3'
]

# 2. 범주형 변수 (One-Hot Encoding 대상)
CORE_CATEGORICAL_FEATURES = ['gender', 'region', 'income_band', 'card_grade']

def preprocess_data(df_input: pd.DataFrame) -> pd.DataFrame:
    df_temp = df_input.copy()

    # [Step 1] 파생변수 생성 (계산에 필요한 원본 컬럼들이 df_input에 있어야 함)
    df_temp['recent_3m_spent'] = df_temp['spent_m1'] + df_temp['spent_m2'] + df_temp['spent_m3']
    df_temp['past_3m_spent'] = df_temp['spent_m4'] + df_temp['spent_m5'] + df_temp['spent_m6']
    df_temp['spent_change_ratio'] = df_temp['recent_3m_spent'] / (df_temp['past_3m_spent'] + 1)
    

    # [Step 2] 데이터 필터링 (고객 ID + 수치형 핵심 피처 + 범주형 피처)
    # 함수 밖에서 정의한 리스트를 사용하여 연결성을 확보합니다.
    features_to_use = ['customer_id'] + HIGH_IMPORTANCE_FEATURES + CORE_CATEGORICAL_FEATURES
    
    # 중요: df_temp에 해당 컬럼이 있는지 확인 후 선택 (오류 방지)
    existing_features = [c for c in features_to_use if c in df_temp.columns]
    df_filtered = df_temp[existing_features].copy()

    # [Step 3] One-Hot Encoding
    df_encoded = pd.get_dummies(df_filtered, columns=CORE_CATEGORICAL_FEATURES, dtype=int)
    
    # [Step 4] 학습 시 컬럼 순서(MODEL_FEATURES)와 완벽 일치시키기
    # 이 과정에서 customer_id는 자연스럽게 사라지거나 fill_value=0으로 처리됨
    df_processed = df_encoded.reindex(columns=MODEL_FEATURES, fill_value=0)
    
    # 만약 MODEL_FEATURES에 customer_id가 포함되어 있지 않다면 여기서 확실히 제거
    if 'customer_id' in df_processed.columns:
        df_processed = df_processed.drop(columns=['customer_id'])
    
    return df_processed

# --- 3. 예측 및 티어 분류 함수 ---

def predict_and_classify(df_new_data: pd.DataFrame) -> pd.DataFrame:
    """새로운 데이터를 받아 확률을 예측하고 위험 티어를 분류합니다."""
    
    # 1. 전처리 및 예측
    df_processed = preprocess_data(df_new_data.copy())
    churn_proba = LOADED_MODEL.predict_proba(df_processed)[:, 1]
    
    df_output = df_new_data[['customer_id']].copy() # customer_id만 추출
    df_output['churn_proba'] = churn_proba

    # 2. 티어 분류 (저장된 임계값 사용)
    T99 = LOADED_THRESHOLDS['T99']
    T95 = LOADED_THRESHOLDS['T95']
    T90 = LOADED_THRESHOLDS['T90']
    
    def assign_risk_tier(proba):
        if proba >= T99:
            return 'Tier 1' # Extreme Risk
        elif proba >= T95:
            return 'Tier 2' # Very High Risk
        elif proba >= T90:
            return 'Tier 3' # High Risk
        else:
            return 'Tier 4' # Low Risk

    df_output['risk_tier'] = df_output['churn_proba'].apply(assign_risk_tier)
    
    return df_output[['customer_id', 'churn_proba', 'risk_tier']]